**DAT405 Introduction to Data Science and AI, 2010-2021, Study Period 2** <br/>
**Assignment 5: Reinforcement learning and Classification** <br/>
**Due Date: Dec 9, 23:59** <br/>

---


**What to submit**
*   **The entire assignment should be submitted through the notebook. No separate file will be accepted.** You can submit either the notebook itself, or a public link to a Google Colab notebook<br/>

*In the notebook:*
*	State your names and how many hours each person spent on the assignment.
*	The solutions and answers to the theoretical and practical problems, including LaTeX math-mode equations, plots and tables etc.
*	All plots/results should be visible such that the notebook does not have to be run. But the code in the notebook should reproduce the plots/results if we choose to do so.<br/>

*Before submitting:*
*   Make sure that your code can run on another computer. That all plots can show on another computer including all your writing. It is good to check if your code can run here: https://colab.research.google.com.

**Self-check**<br/>
Is all the required information included? Have you answered all questions to the best of your ability? Anything else you can easily check? (details, terminology, arguments, clearly stated answers etc.?) Does your notebook run and can reproduce the results, plots and tables?

**Grading**<br/>
Grading will be based on a qualitative assessment of each assignment. It is important to:
*	Present clear arguments
*	Present the results in a pedagogical way
*	Show understanding of the topics (e.g, write a pseudocode) 
*	Give correct solutions
*	Make sure that the code is well commented 

**Again, as mentioned in general guidelines, all code should be written here. And this same ipython notebook file (Assignment5_Reinforcement_Learning.ipynb) should be submitted with answers and code written in it. No separate file will be accepted.** 


# Primer

## Decision Making
The problem of **decision making under uncertainty** (commonly known as **reinforcement learning**) can be broken down into
two parts. First, how do we learn about the world? This involves both the
problem of modeling our initial uncertainty about the world, and that of drawing conclusions from evidence and our initial belief. Secondly, given what we
currently know about the world, how should we decide what to do, taking into
account future events and observations that may change our conclusions?
Typically, this will involve creating long-term plans covering possible future
eventualities. That is, when planning under uncertainty, we also need to take
into account what possible future knowledge could be generated when implementing our plans. Intuitively, executing plans which involve trying out new
things should give more information, but it is hard to tell whether this information will be beneficial. The choice between doing something which is already
known to produce good results and experiment with something new is known
as the **exploration-exploitation dilemma**.

## The exploration-exploitation trade-off

Consider the problem of selecting a restaurant to go to during a vacation. Lets say the
best restaurant you have found so far was **Les Epinards**. The food there is
usually to your taste and satisfactory. However, a well-known recommendations
website suggests that **King’s Arm** is really good! It is tempting to try it out. But
there is a risk involved. It may turn out to be much worse than **Les Epinards**,
in which case you will regret going there. On the other hand, it could also be
much better. What should you do?
It all depends on how much information you have about either restaurant,
and how many more days you’ll stay in town. If this is your last day, then it’s
probably a better idea to go to **Les Epinards**, unless you are expecting **King’s
Arm** to be significantly better. However, if you are going to stay there longer,
trying out **King’s Arm** is a good bet. If you are lucky, you will be getting much
better food for the remaining time, while otherwise you will have missed only
one good meal out of many, making the potential risk quite small.

## Overview
* To make things concrete, we will first focus on decision making under **no** uncertainity, i.e, given we have a world model, we can calculate the exact and optimal actions to take in it. We will first introduce **Markov Decision Process (MDP)** as the world model. Then we give one algorithm (out of many) to solve it.


* Next, we will work through one type of reinforcement learning algorithm called Q-learning. Q-learning is an algorithm for making decisions under uncertainity, where uncertainity is over the possible world model (here MDP). It will find the optimal policy for the **unknown** MDP, assuming we do infinite exploration.

## Markov Decision Process

Markov Decision Process (MDP) provides a mathematical framework for modeling sequential decision making under uncertainty. A MDP consists of five parts: the specific decision times, the state space of the environment/system, the available actions for the decision maker, the rewards, and the transition probabilities between the states.

* Decision epochs: $t={1,2,...,T}$, where $T\leq \infty$
* State space: $S=\{s_1,s_2,...,s_N\}$ of the underlying environment
* Action space $A=\{a_1,a_2,...,a_K\}$ available to the decision maker at each decision epoch
* Reward functions $R_t = r(a_t,s_t,s_{t+1})$ for the current state and action, and the resulting next state
* Transition probabilities $p(s'|s,a)$ that taking action $a$ in state $s$ will lead to state $s'$

At a given decision epoch $t$ and system state $s_t$, the decions maker, or *agent*, chooses an action $a_t$, the system jumps to a new state $s_{t+1}$ according to the transition probability $p(s_{t+1}|s_t,a_t)$, and the agent receives a reward $r_t(s_t,a_t,s_{t+1})$. This process is then repeated for a finite or infinite number of times.

A *decision policy* is a function $\pi: s \rightarrow a$, that gives instructions on what action to choose in each state. A policy can either be *deterministic*, meaning that the action is given for each state, or *randomized* meaning that there is a probability distribution over the set of possible actions. Given a specific policy $\pi$ we can then compute the the *expected total reward* when starting in a given state $s_1 \in S$, which is also known as the *value* for that state, 

$$V^\pi (s_1) = E\left[ \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) {\Large |} s_1\right] = \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) p(s_{t+1} | a_t,s_t)$$ 

where $a_t = \pi(s_t)$. To ensure convergence and to control how much credit to give to future rewards, it is common to introduce a *discount factor* $\gamma \in [0,1]$. For instance, if you think all future rewards should count equally, you would use $\gamma = 1$, while if you only care less about future rewards you would use $\gamma < 1$. The expected total *discounted* reward becomes

$$V^\pi( s_1) = \sum_{t=1}^T \gamma^{t-1} r(s_t,a_t, s_{t+1}) p(s_{t+1} | s_t, a_t) $$

Now, to find the *optimal* policy we want to find the policy $\pi^*$ that gives the highest total reward $V^{\pi^*}(s)$ for all $s\in S$. That is

$$V^{\pi^*}(s) \geq V^\pi(s), s\in S$$

The problem of finding the optimal policy is a _dynamic programming problem_. It turns out that a solution to the optimal policy problem in this context is the *Bellman equation*. The Bellman equation is given by

$$V(s) = \max_{a\in A} \left\{\sum_{s'\in S} p(s'|s,a)( r(s,a,s') +\gamma V(s')) \right\}$$

Thus, it can be shown that if $\pi$ is a policy such that $V^\pi$ fulfills the Bellman equation, then $\pi$ is an optimal policy.

A real world example would be an inventory control system. Your states would be the amount of items you have in stock. Your actions would be the amount to order. The discrete time would be the days of the month. The reward would be the profit.  

A major drawback of MDPs is called the "Curse of Dimensionality". MDPs unfortunately do not scale very well with increasing sets of states or actions.   


* Erik Jergéus 11 h
* Daniel Willim 11 h

## Question 1

In this first question we work with the deterministic MDP, no code is necessary in this part.

Setup:

* The agent starts in state **S**
* The actions possible are **N** (north), **S** (south), **E** (east), and **W** west. 
* Note, that you cannot move outside the grid, thus all actions are not available in every box.
* When reaching **F**, the game ends (absorbing state).
* The numbers in the boxes represent the rewards you receive when moving into that box. 
* Assume no discount in this model: $\gamma = 1$

The reward of a state $r(s=(x, y))$ is given by the values on the grid:
    
| | | |
|----------|----------|---------|
|-1   | 1|**F**|
|0    |-1| 1   |  
|-1   | 0|-1   |  
|**S**|-1| 1   |

Let $(x,y)$ denote the position in the grid, such that $S=(0,0)$ and $F=(2,3)$.

**1a)** What is the optimal path of the MDP above? Is it unique? Submit the path as a single string of directions. E.g. NESW will make a circle.

EENNN, but it's not unique, for example EENNWNE also result in a score of 0. Furthermore one could loop between a state of 1 and -1 indefinitely to get the same reward as well.


**1b)** What is the optimal policy (i.e. the optimal action in each state)?

There are several, but one is:

| | | |
|----------|----------|---------|
|->    |-> |**F**|
|->    |^  |^    |  
|->    |^  |^    |  
|**->**|-> |^    |

**1c)** What is expected total reward for the policy in 1b)?

Since we have 100% probability to succeed in the actions, the optimal policy is to simply take the path EENNN, which results in a total reward of 0.

$$V^\pi (s_1) = E\left[ \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) {\Large |} s_1\right] = \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) p(s_{t+1} | a_t,s_t)$$ 

$$=r(s_{0,0},E,s_{0,1})p(s_{0,1} | E,s_{0,0})+\sum_{t=2}^{T} r(s_t,a_t,s_{t+1}) p(s_{t+1} | a_t,s_t) = -1*1+\sum_{t=2}^{T} r(s_t,a_t,s_{t+1}) p(s_{t+1} | a_t,s_t)...=-1*1+1*1+(-1)*1+1*1+0*1=0$$

## Value Iteration

For larger problems we need to utilize algorithms to determine the optimal policy $\pi^*$. *Value iteration* is one such algorithm that iteratively computes the value for each state. Recall that for a policy to be optimal, it must satisfy the Bellman equation above, meaning that plugging in a given candidate $V^*$ in the right-hand side (RHS) of the Bellman equation should result in the same $V^*$ on the left-hand side (LHS). This property will form the basis of our algorithm. Essentially, it can be shown that repeated application of the RHS to any intial value function $V^0(s)$ will eventually lead to the value $V$ which statifies the Bellman equation. Hence repeated application of the Bellman equation will also lead to the optimal value function. We can then extract the optimal policy by simply noting what actions that satisfy the equation. The process of repeated application of the Bellman equation what we here call the _value iteration_ algorithm.

The value iteration algorithm practically procedes as follows:

```
epsilon is a small value, threshold
for x from i to infinity 
do
    for each state s
    do
        V_k[s] = max_a Σ_s' p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
    end
    if  |V_k[s]-V_k-1[s]| < epsilon for all s
        for each state s,
        do
            π(s)=argmax_a ∑_s′ p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
            return π, V_k 
        end
end

```






**Example:** We will illustrate the value iteration algorithm by going through two iterations. Below is a 3x3 grid with the rewards given in each state. Assume now that given a certain state $s$ and action $a$, there is a probability of 0.8 that that action will be performed and a probability of 0.2 that no action is taken. For instance, if we take action **E** in state $(x,y)$ we will go to $(x+1,y)$ 80 percent of the time (given that that action is available in that state, that is, we stay on the grid), and remain still 20 percent of the time. We will use have a discount factor $\gamma = 0.9$. Let the initial value be $V^0(s)=0$ for all states $s\in S$. 

| | | |  
|----------|----------|---------|  
|0|0|0|
|0|10|0|  
|0|0|0|  


**Iteration 1**: The first iteration is trivial, $V^1(s)$ becomes the $\max_a \sum_{s'} p(s'|s,a) r(s,a,s')$ since $V^0$ was zero for all $s'$. The updated values for each state become

| | | |  
|----------|----------|---------|  
|0|8|0|
|8|2|8|  
|0|8|0|  
  
**Iteration 2**:  
  
Staring with cell (0,0) (lower left corner): We find the expected value of each move:  
Action **S**: 0  
Action **E**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **N**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **W**: 0

Hence any action between **E** and **N** would be best at this stage.

Similarly for cell (1,0):

Action **N**: 0.8( 10 + 0.9 \* 2) + 0.2(0 + 0.9 \* 8) = 10.88 (Action **N** is the maximizing action)  

Similar calculations for remaining cells give us:

| | | |  
|----------|----------|---------|  
|5.76|10.88|5.76|
|10.88|8.12|10.88|  
|5.76|10.88|5.76|  


## Question 2

**2a)** Implement the value iteration algorithm just described here in python, and show the converging optimal value function and the optimal policy for the above 3x3 grid. Hint: use the pseudo-code above as a starting point, but be sure to explain what every line does.



In [ ]:
from pprint import pprint
import numpy as np

# Find max value and it's index 
def arg_max(ls):
  max_v = ls[0]
  max_i = 0
  for i, v in enumerate(ls):
    if v > max_v:
      max_v = v
      max_i = i
    
  return max_i, max_v


def action_value(grid, s, gamma, p): 
  action_val = []
  
  for sx in grid[s][2]:
    action_val.append(p    * (grid[sx][0] + gamma * grid[sx][1]) + 
                     (1-p) * (grid[s][0]  + gamma * grid[s][1]))
  
  i, val = arg_max(action_val)
  act = grid[s][2][i]

  return act, val

# Run value iteration algroithm on grid with start policy pi
def value_iteration(grid, pi, eps=0.3, gamma=0.9, p=0.8):
  while(True):
    done = True
    
    #Save copy of grid to compare if done
    new_grid = grid.copy()

    # Calculate value of each state
    for s in pi.keys(): 
      act, val = action_value(grid, s, gamma, p)

      new_grid[s] = (grid[s][0], val, grid[s][2])

      pi[s] = act

      # End only when all states change less than epsilon
      done = done and abs(new_grid[s][1] - grid[s][1]) < eps
    
    grid = new_grid.copy()

    if done: 
      return pi, grid

# Print Pi and Grip readable
def print_pi_grid(pi, grid):
    p_grid = np.zeros((3,3))
    p_pi = np.zeros((3,3), dtype=tuple)
    
    for (x,y) in pi:
        p_pi[x,y] = pi[(x,y)]

    for (x,y) in grid:
        p_grid[x,y] = grid[(x,y)][1]

    print("Optimal policy (each cell indicates what cell to walk to next):")
    print(p_pi)
    print("Expected value:")
    print(p_grid)

In [ ]:
#state: (Reward for entering state, current expected value, [neighbours])
grid = {
    (0,0): (0, 0, [(0,1), (1,0)              ]), 
    (0,1): (0, 0, [(0,0), (1,1), (0,2)       ]),
    (0,2): (0, 0, [(0,1), (1,2)              ]),
    (1,0): (0, 0, [(0,0), (1,1), (2,0)       ]),
    (1,1): (10,0, [(0,1), (1,0), (2,1), (1,2)]),
    (1,2): (0, 0, [(0,2), (1,1), (2,2)       ]),
    (2,0): (0, 0, [(1,0), (2,1)              ]),
    (2,1): (0, 0, [(1,1), (2,0), (2,2)       ]),
    (2,2): (0, 0, [(1,2), (1,0)              ])
  }

# Inital policy
pi = {
    (0,0):(0,0),
    (0,1):(0,1),
    (0,2):(0,2),
    (1,0):(1,0),
    (1,1):(1,1),
    (1,2):(1,2),
    (2,0):(2,0),
    (2,1):(2,1),
    (2,2):(2,2)
  }

res_pi, res_grid = value_iteration(grid,pi)

print("Intial values 0")
print_pi_grid(res_pi, res_grid)

Intial values 0
Optimal policy (each cell indicates what cell to walk to next):
[[(0, 1) (1, 1) (0, 1)]
 [(1, 1) (0, 1) (1, 1)]
 [(1, 0) (1, 1) (1, 2)]]
Expected value:
[[42.99618556 49.33131372 42.99618556]
 [49.33131372 45.43520995 49.33131372]
 [42.99618556 49.33131372 42.99618556]]


In [ ]:
grid_2 = {
    (0,0): (0, 100, [(0,1), (1,0)              ]), 
    (0,1): (0, 100, [(0,0), (1,1), (0,2)       ]),
    (0,2): (0, 100, [(0,1), (1,2)              ]),
    (1,0): (0, 100, [(0,0), (1,1), (2,0)       ]),
    (1,1): (10,100, [(0,1), (1,0), (2,1), (1,2)]),
    (1,2): (0, 100, [(0,2), (1,1), (2,2)       ]),
    (2,0): (0, 100, [(1,0), (2,1)              ]),
    (2,1): (0, 100, [(1,1), (2,0), (2,2)       ]),
    (2,2): (0, 100, [(1,2), (1,0)              ])
  }

res_pi, res_grid = value_iteration(grid_2,pi)

print("Intial values 100")
print_pi_grid(res_pi, res_grid)

Intial values 100
Optimal policy (each cell indicates what cell to walk to next):
[[(0, 1) (1, 1) (0, 1)]
 [(1, 1) (0, 1) (1, 1)]
 [(1, 0) (1, 1) (1, 2)]]
Expected value:
[[48.22966189 54.56479005 48.22966189]
 [54.56479005 50.66868628 54.56479005]
 [48.22966189 54.56479005 48.22966189]]


**2b)** Explain why the result of 2a) does not depend on the initial value $V_0$.

We can see that the expected value differ somewhat (this depends on if you converge on the expected value from higher or lower numbers), but the optimal path doesn't. Regardless of the initial values we will run the algorithm until we reach values where the path does not differ more than epsilon and the values will converge towards similar values. The reason for this is that the difference between two values of the iteration will always be higher than epsilon before we reach a path that is optimal, or close to it if epsilon is too high. We can however exploit the initial values to get towards the optimal path in fewer iterations, for example if we holistically guess that the values will reach around 40, we can set the initial values to it and thereby reach the optimal path in fewer iterations.

## Reinforcement Learning (RL)
Until now, we understood that knowing the MDP, specifically $p(s'|a,s)$ and $r(a,s,s')$ allows us to efficiently find the optimal policy using the value iteration algorithm. Reinforcement learning (RL) or decision making under uncertainity, however, arises from the question of making optimal decisions without knowing the true world model (the MDP in this case).

So far we have defined the value function for a policy through $V^\pi$. Let's now define the *action-value function*

$$Q^\pi(s,a) = \sum_{s'} p(s'|a,s) [r(a,s,s') + \gamma V^\pi(s')]$$

The value function and the action-value function are directly related through

$$V^\pi (s) = \max_a Q^\pi (s,a)$$

i.e, the value of taking action $a$ in state $s$ and then following the policy $\pi$ onwards. Similarly to the value function, the optimal $Q$-value equation is:

$$Q^*(s,a) = \sum_{s'} p(s'|a,s) [r(a,s
]\,s') + \gamma V^*(s')]$$

and the relationship between $Q^*(s,a)$ and $V^*(s)$ is simply

$$V^*(s) = \max_{a\in A} Q^*(s,a).$$

## Q-learning

Q-learning is a RL-method where the agent learns about its unknown environment (i.e. the MDP is unknown) through exploration. In each time step *t* the agent chooses an action *a* based on the current state *s*, observes the reward *r* and the next state *s'*, and repeats the process in the new state. Q-learning is then a method that allows the agent to act optimally. Here we will focus on the simplest form of Q-learning algorithms, which can be applied when all states are known to the agent, and the state and action spaces are reasonably small. This simple algorithm uses a table of Q-values for each $(s,a)$ pair, which is then updated in each time step using the update rule in step $k+1$

$$Q_{k+1}(s,a) = Q_k(s,a) + \alpha \left( r(s,a) + \gamma \max \{Q_k(s',a')\} - Q_k(s,a) \right) $$ 

where $\gamma$ is the discount factor as before, and $\alpha$ is a pre-set learning rate. It can be shown that this algorithm converges to the optimal policy of the underlying MDP for certain values of $\alpha$ as long as there is sufficient exploration. While a constant $\alpha$ generally does not guarantee us to reach true convergence, we keep it constant at $\alpha=0.1$ for this assignment.

## OpenAI Gym

We shall use already available simulators for different environments (worlds) using the popular OpenAI Gym library. It just implements [different types of simulators](https://gym.openai.com/) including ATARI games. Although here we will only focus on simple ones, such as the [Chain enviroment](https://gym.openai.com/envs/NChain-v0/) illustrated below.
![alt text](https://chalmersuniversity.box.com/shared/static/6tthbzhpofq9gzlowhr3w8if0xvyxb2b.jpg)
The figure corresponds to an MDP with 5 states $S = \{1,2,3,4,5\}$ and two possible actions $A=\{a,b\}$ in each state. The arrows indicate the resulting transitions for each state-action pair, and the numbers correspond to the rewards for each transition.

## Question 3
You are to first familiarize with the framework using its [documentation](http://gym.openai.com/docs/), and then implement the Q-learning algorithm for the Chain enviroment (called 'NChain-v0') using default parameters. Finally print the $Q^*$ table at convergence. Convergence is **not** a constant value, rather a stable plateau with some noise. Take $\gamma=0.95$. You can refer to the Q-learning (frozen lake) Jupyter notebook shown in class, uploaded on Canvas. Hint: start with a small learning rate.


In [ ]:
import gym
import numpy as np
import random
import math

gamma = 0.95
learning_rate = 0.3 # Start with small
epsilon = 0.3
num_episodes = 2000

env = gym.make('NChain-v0')

In [ ]:
Q = np.zeros([5,2]) # states, actions

for _ in range(num_episodes):
  state = env.reset()
  done = False
  while done == False:
    if random.uniform(0, 1) < epsilon:
      action = env.action_space.sample() # Explore
    else:
      action = np.argmax(Q[state,:]) # Exploit learned values
    
    # Perform the action and receive feedback from the environment
    new_state, reward, done, info = env.step(action)
    
    # Learn from feedback
    update = reward + (gamma*np.max(Q[new_state,:])) - Q[state, action]
    Q[state,action] += learning_rate*update 
    state = new_state
print("  Forwards   | Backwards")
print(Q)

  Forwards   | Backwards
[[46.21048802 46.14860464]
 [48.05591004 47.45743688]
 [50.64878291 50.26603536]
 [51.72071218 49.51441623]
 [56.72849585 51.89561751]]


## Question 4

**4a)** Define the MDP corresponding to the Chain environment above and verify that the optimal $Q^*$ value obtained using simple Q-learning is the same as the optimal value function $V^*$ for the corresponding MDP's optimal action. Hint: compare values obtained using value iteration and Q-learning.


In [ ]:
Q = np.zeros([5,2])
grid = {
    (0,0): (2, 0, [(0,0), (1,0)]), 
    (1,0): (0, 0, [(0,0), (2,0)]),
    (2,0): (0, 0, [(0,0), (3,0)]),
    (3,0): (0, 0, [(0,0), (4,0)]),
    (4,0): (0, 0, [(0,0), (4,1)]),#   Going to last position gives 0  points
    (4,1): (10,0, [(0,0), (4,1)]) # Staying at last position gives 10 points
}
pi = {
      (0,0):(0,0),
      (1,0):(0,0),
      (2,0):(0,0),
      (3,0):(0,0),
      (4,0):(0,0),
      (4,1):(0,0)
}

def print_chain_env(pi, grid):
  p_pi = np.zeros(6, dtype=tuple)
  p_grid = np.zeros(6)

  for x,y in pi:
    p_pi[x+y] = pi[(x,y)]
  
  for x,y in grid:
    p_grid[x+y] = grid[(x,y)][1]

  print("Optimal policy")
  print(p_pi)
  print("Expected value")
  print(p_grid)

# same epsilon & gamma
res_pi, res_grid = value_iteration(grid,pi,eps=epsilon,gamma=gamma, p=0.8)

print("Second to last state is going into last link on chain (recieves no reward). Last state is staying in last link (recieves large reward for action)")
print_chain_env(res_pi,res_grid)

Second to last state is going into last link on chain (recieves no reward). Last state is staying in last link (recieves large reward for action)
Optimal policy
[(1, 0) (2, 0) (3, 0) (4, 0) (4, 1) (4, 1)]
Expected value
[148.06794718 157.64588701 168.38027024 179.82086289 192.01412611
 194.48326191]


In the value iteration algorithm we see that the difference between staying in the first location and going forward is not huge. Therefore the Q-learning algorithm sometimes doesn't find that the optimal path is to explore. However in almost all cases Q-learning finds the proper path, especially if we have a relatively high exploration rate. The values obtained are not the exact same values, but they differ by roughly the same proportion between each state (in value iteration vs explore action in Q-learning).

**4b)** What is the importance of exploration in RL? Explain with an example.

In for example our N-Chain (N=5) it does not benefit the model to go to state 2,3 or 4. Without exploration it would therefore be the best for the model to simply stay at state 1 (if the default action is to stay). However since we have exploration in our model it finds that it is actually more valuable to go towards state 5 (in most iterations). Without exploration the incentive to look for the value in state 5 would not exist and as such would never be found. We can even see in our example that our exploration rate might be a bit low, as it seems to have trouble between deciding to stay or go forwards in the first state. As a sidenote, if the default action would be to go forwards, the algorithm still wouldn't be optimal for all versions of N-Chain. For example if we increase the chain to be a lot longer it would be better to stay at the start, but a RL algorithm without exploration would not explore the possibility that staying still would be optimal.

## Question 5

**5a)** Give a summary of how a decision tree works and how it extends to random forests.

Decision tree is a classification method that in each step of the algorithm splits the data into two or more branches according to a certain feature. This will in the end build a tree where the leaf nodes are the different classes of data. 

For example, let’s consider 6 farm animals (2 dogs, 1 pig, 3 chickens) that should be classified. In the first step we cloud split the data with the question "*Have 4 legs*". Now we have 2 branches one there all animals have 4 legs (dogs and pigs) and one there they do not have 4 legs (chickens). We now split the first group again with the question "*Have long fur*" splitting the group into two new branches one with the dog and one with the pigs. This example was rather trivial, but it highlights how decision trees work. 

But decision trees have some problems, for one they are very sensitive to difference in input data. For example, if one of the dogs in the previous example would only have 3 legs it would have ended up with the chickens instead and not been able to be grouped with the other dog. 

One method to mitigate the problems with decision trees are random forest, where you create N different decision trees on different subsets of the data and feature set. New data is then classified by majority vote in the forest. 


**5b)** Explain what makes reinforcement learning different from supervised learning tasks such as regression or classification.

In supervised learning we have a large set of label data and want to find patterns in the data in order to be able to predict the label of a new datapoint. But there are applications that does not lend themselves to this approach for example there might not be any good datasets or the AI need to adapt to what's happening. A pratical example of this is creating AIs that play games like StarCraft II or Go.

Instead for applications like this reinforcement learning is often used where you have a enviroment (the go board), a state (where each piece is located on the board), a set of possible actions (what pieces can be moved where) and an agent (the AI who decides what action to take). You then let the agent decide what action from the action set to take depending on the state and the enviroment then returns a new state, action set and a reward depending on if the action was good, neutral or bad. The agent is then trained to maximize the reward. With this approach you don't need large sets of data, you only need a enviroment, a set of possible action depending on the state and some rewardfunction and the training data is created as the agent takes actions on the eviroment. 


# References
Primer/text based on the following references:
* http://www.cse.chalmers.se/~chrdimi/downloads/book.pdf
* https://github.com/olethrosdc/ml-society-science/blob/master/notes.pdf